## Installing and Imports libraries

In [15]:
#!pip3 install scikit-image

In [16]:
# Imports
#import random
import numpy as np
import pandas as pd
import os
import sys
#import codecs
import PIL
import cv2
import re
#import string

#import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords, wordnet
from pymorphy2 import MorphAnalyzer

# Input data files are available in the read-only "../input/" directory

# ML libraries
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# keras
import tensorflow as tf
from tensorflow import keras
import keras.layers as L
from keras.models import Model, Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import efficientnet.tfkeras as efn
#from keras_bert import load_trained_model_from_checkpoint
import albumentations

# plt
import matplotlib.pyplot as plt

# set not big default graphs
from pylab import rcParams
rcParams['figure.figsize'] = 8, 4
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

print('Python         :', sys.version.split('\n')[0])
print('Numpy          :', np.__version__)
print('Tensorflow     :', tf.__version__)
print('Albumentations :', albumentations.__version__)

Python         : 3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]
Numpy          : 1.24.2
Tensorflow     : 2.9.1
Albumentations : 1.3.0


In [17]:
# Imports of local libs
sys.path.insert(1, '../')

from utils.paths import create_paths
from utils.read_config import config_reader

from utils.models import MyNaiveModel
from utils.nbook_functions import mape_func, create_preproc_data, define_cat_boost
from utils.eda_functions import make_unzip, pre_an_types_feat, prints_types_feat

In [18]:
# read defined params
config = config_reader('../config/data_config.json')
paths = create_paths(config)

# fix RANDOM_SEED for repeating results
RANDOM_SEED = config.RANDOM_SEED
np.random.seed(RANDOM_SEED)

## Reading Data

In [19]:
# unzip data
is_unzip = False
make_unzip(paths.PATH_TO_ZIP, paths.PATH_DATA, is_unzip=is_unzip)

Zip-archive "sf-dst-car-price-prediction-part2.zip" no need to unpack, to unpack change value of param "is_unzip" to "True"


In [20]:
# Reading '.cvs' data files
train = pd.read_csv(os.path.join(paths.PATH_DATA, 'train.csv'))
test = pd.read_csv(os.path.join(paths.PATH_DATA, 'test.csv'))
sample_submission = pd.read_csv(os.path.join(paths.PATH_DATA, 'sample_submission.csv'))

# Let's look at the train data set columns
print(train.columns)

Index(['bodyType', 'brand', 'color', 'description', 'engineDisplacement',
       'enginePower', 'fuelType', 'mileage', 'modelDate', 'model_info', 'name',
       'numberOfDoors', 'price', 'productionDate', 'sell_id',
       'vehicleConfiguration', 'vehicleTransmission', 'Владельцы', 'Владение',
       'ПТС', 'Привод', 'Руль'],
      dtype='object')


In [21]:
train.sample(2)

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_info,...,price,productionDate,sell_id,vehicleConfiguration,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль
1772,седан,MERCEDES,чёрный,Состояние нового автомобиля.,5.5 LTR,585 N12,бензин,2900,2013,S_KLASSE_AMG,...,7100000.0,2016,1100165184,SEDAN AUTOMATIC 5.5,автоматическая,1 владелец,2 года и 6 месяцев,Оригинал,полный,Левый
4813,хэтчбек 5 дв.,AUDI,красный,При покупке автомобиля:\n► Возможна покупка ав...,2.0 LTR,150 N12,бензин,224000,2004,A3,...,389000.0,2005,1099744048,HATCHBACK_5_DOORS AUTOMATIC 2.0,автоматическая,3 или более,NaN,Оригинал,передний,Левый


In [22]:
# Comparison of the number of columns in training and test datasets
print(f'Num train cols: {len(train.columns)}, ', f'Num test cols: {len(test.columns)}')
print(f'Col not in test: {set(train.columns) - set(test.columns)}')

# Col with name 'price' - its a target

Num train cols: 22,  Num test cols: 21
Col not in test: {'price'}


In [23]:
# print info about train dataset
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6682 entries, 0 to 6681
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              6682 non-null   object 
 1   brand                 6682 non-null   object 
 2   color                 6682 non-null   object 
 3   description           6682 non-null   object 
 4   engineDisplacement    6682 non-null   object 
 5   enginePower           6682 non-null   object 
 6   fuelType              6682 non-null   object 
 7   mileage               6682 non-null   int64  
 8   modelDate             6682 non-null   int64  
 9   model_info            6682 non-null   object 
 10  name                  6682 non-null   object 
 11  numberOfDoors         6682 non-null   int64  
 12  price                 6682 non-null   float64
 13  productionDate        6682 non-null   int64  
 14  sell_id               6682 non-null   int64  
 15  vehicleConfiguration 

In [24]:
# Automatic types of test features pre-analisys
feat_bin, feat_cat, feat_cat_large, feat_other = pre_an_types_feat(test)
prints_types_feat(feat_bin, feat_cat, feat_cat_large, feat_other)

Binary features: 
['ПТС', 'Руль'] 
 ----------------------------------------
Categorical features with 2 < num_cat <= 20: 
['bodyType', 'brand', 'color', 'fuelType', 'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'Привод'] 
 ----------------------------------------
Categorical or num features with unique num of values: 20 < num_cat <= 50: 
['engineDisplacement', 'modelDate', 'productionDate'], 
 ----------------------------------------
Num or other types of features with unique num of values > 50: 
['description', 'enginePower', 'mileage', 'model_info', 'name', 'sell_id', 'vehicleConfiguration', 'Владение']


### Types of features 


<details>
<summary> Display types of all used features </summary> <br>

* bodyType - categorical
* brand - categorical
* color - categorical
* description - text
* engineDisplacement - numeric, represented as text
* enginePower - numeric, represented as text
* fuelType - categorical
* mileage - numeric
* modelDate - numeric or categorical with large num categories
* model_info - categorical
* name - categorical, needs to reduce nums of unique values
* numberOfDoors - categorical
* price - numeric, target
* productionDate - numeric or categorical with large num categories
* sell_id - text, link to images (images are available at the addresses based on sell_id)
* vehicleConfiguration - don't use (cause it's a combination of other columns)
* vehicleTransmission - categorical
* Владельцы - categorical
* Владение - numeric, represented as text
* ПТС - binary
* Привод - categorical
* Руль - binary

Note: Conclusions were done using manual analysis of train and test DataFrames. 
</details>

## Model 1: the "naive" model
This Model 1 will predict the average price by model name and year of manufacture of car.  
We will compare other models with it.

In [25]:
# splitting of data by train and test
data_train, data_test = train_test_split(train, test_size=0.13, shuffle=True, random_state=RANDOM_SEED)

In [26]:
# predicts of "naive" model
predicts = MyNaiveModel(data_train, data_test).predicts

# evaluating the accuracy (mape metric)
print("The accuracy of the naive model according to the MAPE metric: ",
      f"{(mape_func(data_test['price'], predicts['price'].values))*100:0.2f}%")

The accuracy of the naive model according to the MAPE metric:  18.02%


# Model 2: CatBoostRegressor

In [27]:
X_sub, X_train, X_test, y_train, y_test = create_preproc_data(config, paths)

tabular preproc done
data preproc finish


In [28]:
model = define_cat_boost(config)

model.fit(X_train, y_train,
          eval_set=(X_test, y_test),
          verbose_eval=config.cat_verbose_eval,
          use_best_model=True,
          #metric_period
          #plot=True
          )

test_predict_catboost = model.predict(X_test)
print(f"TEST mape: {(mape_func(y_test, test_predict_catboost))*100:0.2f}%")

0:	learn: 0.6232343	test: 0.6117073	best: 0.6117073 (0)	total: 52.4ms	remaining: 1h 27m 16s
100:	learn: 0.2072257	test: 0.2205924	best: 0.2205924 (100)	total: 4.62s	remaining: 1h 16m 14s
200:	learn: 0.1379908	test: 0.1564758	best: 0.1564758 (200)	total: 8.66s	remaining: 1h 11m 39s
300:	learn: 0.1208962	test: 0.1423897	best: 0.1423897 (300)	total: 12.3s	remaining: 1h 7m 57s
400:	learn: 0.1130210	test: 0.1378327	best: 0.1378261 (398)	total: 16.1s	remaining: 1h 6m 39s
500:	learn: 0.1073971	test: 0.1340897	best: 0.1340897 (500)	total: 21.1s	remaining: 1h 9m 46s
600:	learn: 0.1032988	test: 0.1312696	best: 0.1312696 (600)	total: 26.9s	remaining: 1h 14m 17s
700:	learn: 0.0997892	test: 0.1292035	best: 0.1292035 (700)	total: 32.1s	remaining: 1h 15m 44s
800:	learn: 0.0969430	test: 0.1278225	best: 0.1278225 (800)	total: 36.7s	remaining: 1h 15m 49s
900:	learn: 0.0943352	test: 0.1263600	best: 0.1263600 (900)	total: 41.1s	remaining: 1h 15m 20s
1000:	learn: 0.0921115	test: 0.1255696	best: 0.1255688 (

### Submission

In [15]:
sub_predict_catboost = model.predict(X_sub)
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('../catboost_submission.csv', index=False)

# Model 3: Tabular NN

## Simple Dense NN

In [ ]:
X_sub, X_train, X_test, y_train, y_test = create_preproc_data(config, paths) #X, y, 
list_inds = [X_train.index, X_test.index, X_sub.index]

In [ ]:
def callbacks(lr):
    checkpoint = ModelCheckpoint('../working/best_model.hdf5' , monitor='val_MAPE', 
                                 verbose=1, mode='min', save_best_only=True)
    earlystop = EarlyStopping(monitor='val_MAPE', patience=85, restore_best_weights=True,)
    reduce_lr = ReduceLROnPlateau(monitor='val_MAPE', factor=0.5, patience=25, verbose=1, 
                                  min_lr=lr/1000, mode='min')
    
    return [checkpoint, earlystop, reduce_lr]

In [ ]:
model = Sequential()
model.add(L.Dense(1024, input_dim=X_train.shape[1], activation="relu"))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.5))
model.add(L.Dense(512, activation="relu"))
model.add(L.Dropout(0.25))
model.add(L.Dense(256, activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation="linear"))

print(model.summary())

# Compile model
LR = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate=LR,
                                     beta_1=0.992, 
                                     beta_2=0.999,
                                     epsilon=2e-06,
                                     amsgrad=False,
                                    )
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

callbacks_list = callbacks(lr=LR)

# fit
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=500, 
                    validation_data=(X_test, y_test),
                    callbacks=callbacks_list,
                    verbose=1,
                    )

# plot history
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

# save_model
model.load_weights('../working/best_model.hdf5')
model.save_weights('../working/best_model_nn1.hdf5')
model.save('../working/nn_1.hdf5')

# test predict
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

In [ ]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:,0]
sample_submission.to_csv('nn1_submission.csv', index=False)

# Model 4: NLP + Multiple Input

## Work with data.description

In [ ]:
X, y, X_sub, X_train, X_test, y_train, y_test, data = create_data()

In [ ]:
text_train_sequences, text_test_sequences, text_sub_sequences, tokenize = data_descr_to_nlp()
# текст после преобразования
#print(text_train.iloc[0])
#print(text_train_sequences[0])

### Создаем модель MultiNN: SimpleNN + NLP

### RNN NLP

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.BatchNormalization())
model_nlp.add(L.LSTM(1024, return_sequences=True))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.LSTM(512,))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(256, activation="relu"))
model_nlp.add(L.Dropout(0.5))

### MLP

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(1024, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.BatchNormalization())
model_mlp.add(L.Dropout(0.25))
model_mlp.add(L.Dense(512, activation="relu"))
model_mlp.add(L.Dropout(0.5))
#model_mlp.add(L.Dense(256, activation="relu"))
#model_mlp.add(L.Dropout(0.5))

### Multiple Inputs NN

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
#head = L.BatchNormalization()(head)
head = L.Dropout(0.5)(head)
head = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head)

### Fit

In [ ]:
def callbacks(lr):
    checkpoint = ModelCheckpoint('../working/best_model_m_inps.hdf5' , monitor='val_MAPE', 
                                 verbose=1, mode='min', save_best_only=True)
    earlystop = EarlyStopping(monitor='val_MAPE', patience=10, restore_best_weights=True,)
    
    return [checkpoint, earlystop]

In [ ]:
LR = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])
callbacks_list = callbacks(lr=LR)

In [ ]:
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, 
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list,
                    verbose=1
                   )

plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show();

In [ ]:
model.load_weights('../working/best_model_m_inps.hdf5')
model.save_weights('../working/best_model_m_inps_st1.hdf5')
model.save('../working/nn_mlp_nlp_st1.hdf5')

test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
model.load_weights('../working/best_model_m_inps_st1.hdf5')

In [ ]:
LR = 0.005 
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])
callbacks_list = callbacks(lr=LR)

history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, 
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list,
                    verbose=1
                   )

model.load_weights('../working/best_model_m_inps.hdf5')
model.save_weights('../working/best_model_m_inps_st2.hdf5')
model.save('../working/nn_mlp_nlp_st2.hdf5')

test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
model.load_weights('../working/best_model_m_inps_st2.hdf5')

In [ ]:
LR = 0.002 
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])
callbacks_list = callbacks(lr=LR)

history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, 
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list,
                    verbose=1
                   )

model.load_weights('../working/best_model_m_inps.hdf5')
model.save_weights('../working/best_model_m_inps_st3.hdf5')
model.save('../working/nn_mlp_nlp_st3.hdf5')

test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
model.load_weights('../working/best_model_m_inps_st3.hdf5')

In [ ]:
LR = 0.0002
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])
callbacks_list = callbacks(lr=LR)

history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, 
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list,
                    verbose=1
                   )

model.load_weights('../working/best_model_m_inps.hdf5')
model.save_weights('../working/best_model_m_inps_st4.hdf5')
model.save('../working/nn_mlp_nlp_st4.hdf5')

test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
model.load_weights('../working/best_model_m_inps_st4.hdf5')

In [ ]:
LR = 0.0001 
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])
callbacks_list = callbacks(lr=LR)

history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, 
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list,
                    verbose=1
                   )

model.load_weights('../working/best_model_m_inps.hdf5')
model.save_weights('../working/best_model_m_inps_st5.hdf5')
model.save('../working/nn_mlp_nlp_st5.hdf5')

test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
model.load_weights('../working/best_model_m_inps_st5.hdf5')

In [ ]:
LR = 0.0000005 
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])
callbacks_list = callbacks(lr=LR)

history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, 
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list,
                    verbose=1
                   )

model.load_weights('../working/best_model_m_inps.hdf5')
model.save_weights('../working/best_model_m_inps_st6.hdf5')
model.save('../working/nn_mlp_nlp_st6.hdf5')

test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
model.load_weights('../working/best_model_m_inps_st6.hdf5')

In [ ]:
LR = 0.00000005 
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])
callbacks_list = callbacks(lr=LR)

history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500, 
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list,
                    verbose=1
                   )

model.load_weights('../working/best_model_m_inps.hdf5')
model.save_weights('../working/best_model_m_inps_st7.hdf5')
model.save('../working/nn_mlp_nlp_st7.hdf5')

test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

In [ ]:
model.load_weights('../working/best_model_m_inps_st7.hdf5')

In [ ]:
sub_predict_nn2 = model.predict([text_sub_sequences, X_sub])
#sample_submission['price'] = sub_predict_nn2[:,0]
#sample_submission.to_csv('nn2_submission.csv', index=False)

sample_submission['price'] = np.round(sub_predict_nn2[:,0],-4) # округляем до 4 знаков до запятой!!!
sample_submission.to_csv('nn2_submission_wth_round.csv', index=False)

# Model 5: Добавляем картинки

In [ ]:
X, y, X_sub, X_train, X_test, y_train, y_test, data = create_data()
print(X.shape, y.shape, X_sub.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape, data.shape)

text_train_sequences, text_test_sequences, text_sub_sequences, tokenize = data_descr_to_nlp()
print(text_train_sequences.shape, text_test_sequences.shape, text_sub_sequences.shape)

In [ ]:
# смотрим примеры изображений и таргет (цена)
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [ ]:
# пример картинки и ее размеры, чтобы понимать, как лучше обрабатывать и сжимать картинки
image = PIL.Image.open(DATA_DIR+'img/img/' + '35782220.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

In [ ]:
size = (320, 240)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

In [ ]:
augmentation = albumentations.Compose([
    albumentations.CLAHE(p=0.25, clip_limit=(1, 10), tile_grid_size=(10, 10)),
    albumentations.ChannelShuffle(p=0.25),
    #albumentations.ElasticTransform(p=0.25, alpha=1.0, sigma=10, alpha_affine=10, 
    #                                interpolation=1, border_mode=1, value=(0, 0, 0), 
    #                                mask_value=None, approximate=False),
    albumentations.Equalize(p=0.25, mode='cv', by_channels=True),
    albumentations.GaussNoise(p=0.25, var_limit=(10.0, 500.0), mean=-10),
    #albumentations.GridDistortion(p=0.25, num_steps=15, distort_limit=(-0.3, 0.3), 
    #                              interpolation=3, border_mode=1, value=(0, 0, 0), 
    #                              mask_value=None),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.HueSaturationValue(p=0.5, hue_shift_limit=(-20, 20), 
                                      sat_shift_limit=(-20, 20), val_shift_limit=(-20, 20)),
    #albumentations.ISONoise(p=0.5, intensity=(0.1, 0.4), color_shift=(0.01, 0.3)),
    albumentations.MotionBlur(p=0.25, blur_limit=(3, 7)),
    #albumentations.OpticalDistortion(p=0.25, distort_limit=(-0.3, 0.3), 
    #                                 shift_limit=(-0.2, 0.2), interpolation=2, 
    #                                 border_mode=1, value=(0, 0, 0), mask_value=None),
    albumentations.RGBShift(p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
        ],p=0.5),
    albumentations.RandomGamma(p=0.25, gamma_limit=(100, 130), eps=1e-07),
    #albumentations.RandomSnow(p=0.25, snow_point_lower=0.25, snow_point_upper=0.75, 
    #                          brightness_coeff=1.2),
    albumentations.Rotate(p=0.5, limit=(-5, 5), interpolation=2, border_mode=2)
    ])

In [ ]:
# пример применения аугментаций
img_1 = cv2.imread("../input/sf-dst-car-price-prediction-part2/img/img/35782220.jpg")
plt.figure(figsize=(12,8))
for i in range(0,6):
    x = augmentation(image = img_1)['image']
    plt.subplot(3,3, i+1)
    plt.imshow(x)
plt.show()

In [ ]:
def make_augmentations(images):
    print('применение аугментаций', end = '')
    augmented_images = np.empty(images.shape)
    for i in range(images.shape[0]):
        if i % 200 == 0:
            print('.', end = '')
        augment_dict = augmentation(image = images[i])
        augmented_image = augment_dict['image']
        augmented_images[i] = augmented_image
    print('')
    return augmented_images

In [ ]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), 
                                  maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
test_dataset.__iter__().__next__();
sub_dataset.__iter__().__next__();

In [ ]:
def callbacks(lr):
    checkpoint = ModelCheckpoint('../working/best_model_all.hdf5' , monitor='val_MAPE', 
                                 verbose=1, mode='min', save_best_only=True)
    earlystop = EarlyStopping(monitor='val_MAPE', patience=15, restore_best_weights=True,)
    reduce_lr = ReduceLROnPlateau(monitor='val_MAPE', factor=0.3, patience=5, verbose=1, 
                                  min_lr=lr/1000, mode='min')
    
    return [checkpoint, earlystop, reduce_lr]

In [ ]:
# Step 1 Model 5 Fine_tunning - train only head & begin from LR = 0.01
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(
    weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_model.trainable = False

efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)
efficientnet_output = L.Dense(512, activation="relu")(efficientnet_output)
efficientnet_output = L.Dropout(0.5)(efficientnet_output)

In [ ]:
# tabular model
tabular_model = Sequential()
tabular_model.add(L.Dense(1024, input_dim=X_train.shape[1], activation="relu"))
tabular_model.add(L.Dropout(0.25))
tabular_model.add(L.Dense(512, activation="relu"))
tabular_model.add(L.Dropout(0.5))

In [ ]:
# nlp model
nlp_model  = Sequential()
nlp_model.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
nlp_model.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
nlp_model.add(L.LSTM(1024, return_sequences=True))
nlp_model.add(L.Dropout(0.25))
nlp_model.add(L.LSTM(512,))
nlp_model.add(L.Dropout(0.5))
nlp_model.add(L.Dense(256, activation="relu"))
nlp_model.add(L.Dropout(0.5))

In [ ]:
# объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dropout(0.5)(head)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)
#model.summary()

In [ ]:
# model compile & fit
LR = 0.01
optimizer = tf.keras.optimizers.Adam(LR)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

callbacks_list = callbacks(lr=LR)

history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data = test_dataset.batch(30),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model_all.hdf5')
model.save_weights('../working/best_model_all_st1.hdf5')
#model.save('../working/nn_final_st1.hdf5')

In [ ]:
# Step 2 Model 5 Fine_tunning - train ~33% last layers of EfficientNetB3 & head & begin from LR = 0.005
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(
    weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))

efficientnet_model.trainable = True
fine_tune_at = int(len(efficientnet_model.layers)//1.5)
for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False

efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)
efficientnet_output = L.Dense(512, activation="relu")(efficientnet_output)
efficientnet_output = L.Dropout(0.5)(efficientnet_output)

#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dropout(0.5)(head)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)

LR = 0.005
optimizer = tf.keras.optimizers.Adam(LR)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

callbacks_list = callbacks(lr=LR)

model.load_weights('../input/weightsl-simple-albu-aft-st1/best_model_all_st1(simple albu).hdf5')

history = model.fit(train_dataset.batch(20),
                    epochs=100,
                    validation_data = test_dataset.batch(20),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model_all.hdf5')
model.save_weights('../working/best_model_all_st2.hdf5')
#model.save('../working/nn_final_st2.hdf5')

In [ ]:
# Step 3 Model 5 Fine_tunning - train ~50% last layers of EfficientNetB3 & head & begin from LR = 0.001
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(
    weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))

efficientnet_model.trainable = True
fine_tune_at = int(len(efficientnet_model.layers)//2)
for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False

efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)
efficientnet_output = L.Dense(512, activation="relu")(efficientnet_output)
efficientnet_output = L.Dropout(0.5)(efficientnet_output)

#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dropout(0.5)(head)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)

LR = 0.001
optimizer = tf.keras.optimizers.Adam(LR)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

callbacks_list = callbacks(lr=LR)

model.load_weights('./best_model_all_st2.hdf5')

history = model.fit(train_dataset.batch(20),
                    epochs=100,
                    validation_data = test_dataset.batch(20),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model_all.hdf5')
model.save_weights('../working/best_model_all_st3.hdf5')
#model.save('../working/nn_final_st3.hdf5')

In [ ]:
# Step 4_1 Model 5 Fine_tunning - train ~75% last layers of EfficientNetB3 & head & begin from LR = 0.0005
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(
    weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))

efficientnet_model.trainable = True
fine_tune_at = int(len(efficientnet_model.layers)//4)
for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False

efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)
efficientnet_output = L.Dense(512, activation="relu")(efficientnet_output)
efficientnet_output = L.Dropout(0.5)(efficientnet_output)

#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dropout(0.5)(head)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)

LR = 0.0005
optimizer = tf.keras.optimizers.Adam(LR)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

callbacks_list = callbacks(lr=LR)

model.load_weights('../working/best_model_all_st3.hdf5')

history = model.fit(train_dataset.batch(20),
                    epochs=100,
                    validation_data = test_dataset.batch(20),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model_all.hdf5')
model.save_weights('../working/best_model_all_st4_1.hdf5')
#model.save('../working/nn_final_st4_1.hdf5')

In [ ]:
# Step 4_2 Model 5 Fine_tunning - train ~75% last layers of EfficientNetB3 & head & begin from LR = 0.0001
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(
    weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))

efficientnet_model.trainable = True
fine_tune_at = int(len(efficientnet_model.layers)//4)
for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False

efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)
efficientnet_output = L.Dense(512, activation="relu")(efficientnet_output)
efficientnet_output = L.Dropout(0.5)(efficientnet_output)

#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dropout(0.5)(head)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)

LR = 0.0001
optimizer = tf.keras.optimizers.Adam(LR)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

callbacks_list = callbacks(lr=LR)

model.load_weights('../working/best_model_all_st4_1.hdf5')

history = model.fit(train_dataset.batch(20),
                    epochs=100,
                    validation_data = test_dataset.batch(20),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model_all.hdf5')
model.save_weights('../working/best_model_all_st4_2.hdf5')
#model.save('../working/nn_final_st4_2.hdf5')

In [ ]:
# Step 5_1 Image model Fine_tunning - train all layers of EfficientNetB3 & head & begin from LR = 0.00005
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(
    weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_model.trainable = True

efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)
efficientnet_output = L.Dense(512, activation="relu")(efficientnet_output)
efficientnet_output = L.Dropout(0.5)(efficientnet_output)

#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dropout(0.5)(head)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)

LR = 0.00005
optimizer = tf.keras.optimizers.Adam(LR)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

callbacks_list = callbacks(lr=LR)

model.load_weights('../working/best_model_all_st4_2.hdf5')

history = model.fit(train_dataset.batch(20),
                    epochs=100,
                    validation_data = test_dataset.batch(20),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model_all.hdf5')
model.save_weights('../working/best_model_all_st5_1.hdf5')
#model.save('../working/nn_final_st5_1.hdf5')

In [ ]:
model.load_weights('../working/best_model_all_st5_1.hdf5')
# предсказание без округления
sub_predict_nn3 = model.predict(sub_dataset.batch(20))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission_st5_1.csv', index=False)

In [ ]:
# предсказание с округлением
sub_predict_nn3 = model.predict(sub_dataset.batch(20))
sample_submission['price'] = np.round(sub_predict_nn3[:,0],-4)
sample_submission.to_csv('nn3_submission_r4_st5_1.csv', index=False)

In [ ]:
## step 6 Model 5 Fine_tunning - train all layers of EfficientNetB3 & head & change size of picture
size = (448, 336)

def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

In [ ]:
# more simple
augmentation = albumentations.Compose([
    albumentations.CLAHE(p=0.25, clip_limit=(1, 10), tile_grid_size=(10, 10)),
    albumentations.ChannelShuffle(p=0.25),
    albumentations.Equalize(p=0.25, mode='cv', by_channels=True),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
        ],p=0.5),
    albumentations.Rotate(p=0.5, limit=(-5, 5), interpolation=2, border_mode=2)
    ])

In [ ]:
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_model.trainable = True

efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)
efficientnet_output = L.Dense(512, activation="relu")(efficientnet_output)
efficientnet_output = L.Dropout(0.5)(efficientnet_output)

#объединяем выходы трех нейросетей
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dropout(0.5)(head)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)

LR = 0.000001
optimizer = tf.keras.optimizers.Adam(LR)
model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

callbacks_list = callbacks(lr=LR)

model.load_weights('../working/best_model_all_st5_1.hdf5')

history = model.fit(train_dataset.batch(10),
                    epochs=100,
                    validation_data = test_dataset.batch(10),
                    callbacks=callbacks_list
                   )

In [ ]:
model.load_weights('../working/best_model_all.hdf5')
model.save_weights('../working/best_model_all_st6.hdf5')
#model.save('../working/nn_final_st6.hdf5')

In [ ]:
model.load_weights('../working/best_model_all_st6.hdf5')

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(10))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [ ]:
# предсказание без округления
sub_predict_nn3 = model.predict(sub_dataset.batch(10))
sample_submission['price'] = sub_predict_nn3[:,0]
sample_submission.to_csv('nn3_submission_st6.csv', index=False)

In [ ]:
# предсказание с округлением
sub_predict_nn3_rounded = model.predict(sub_dataset.batch(10))
sample_submission['price'] = np.round(sub_predict_nn3_rounded[:,0], -4)
sample_submission.to_csv('nn3_submission_r4_st6.csv', index=False)

# Blend

In [ ]:
sub_predict_catboost = np.array(pd.read_csv('../input/best-cat-boost/catboost_submission(2).csv')['price'])
sub_predict_nn1 = np.array(pd.read_csv('../input/best-sub-nn1/nn1_submission(2).csv')['price'])
sub_predict_nn2 = np.array(pd.read_csv('../input/best-sub-nn2/nn2_submission(2)(1).csv')['price'])
sub_predict_nn3 = np.array(pd.read_csv('../input/best-sub-nn3/nn3_submission_st5_1(1).csv')['price'])

In [ ]:
blend_sub_predict = (sub_predict_catboost + sub_predict_nn1 + sub_predict_nn2 + sub_predict_nn3)/4
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)

In [ ]:
sub_predict_catboost_r4 = np.array(pd.read_csv('../input/best-cat-boost/catboost_submission(2).csv')['price'])
sub_predict_nn1_r4 = np.array(pd.read_csv('../input/best-sub-nn1/nn1_submission(2).csv')['price'])
sub_predict_nn2_r4 = np.array(pd.read_csv('../input/best-sub-nn2/nn2_submission(2)(1).csv')['price'])
sub_predict_nn3_r4 = np.array(pd.read_csv('../input/best-sub-nn3/nn3_submission_st5_1(1).csv')['price'])

In [ ]:
blend_sub_predict_rounded = (sub_predict_catboost_r4 + sub_predict_nn1_r4 + sub_predict_nn2_r4 + sub_predict_nn3_r4)/4
sample_submission['price'] = blend_sub_predict_rounded
sample_submission.to_csv('blend_submission_sum_rounded.csv', index=False)

In [ ]:
blend_sub_predict_w_out_cat = (sub_predict_nn1 + sub_predict_nn2 + sub_predict_nn3)/3
sample_submission['price'] = blend_sub_predict_w_out_cat
sample_submission.to_csv('blend_submission_w_out_cat.csv', index=False)